In [ ]:
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from chess_game import CNNModel
from torch.utils.data import DataLoader, TensorDataset, random_split

# --------------------------
# Training Hyperparameters
# --------------------------
LEARNING_RATE = 1e-3
NUM_EPOCHS = 50
PATIENCE = 5
BATCH_SIZE = 512

# --------------------------
# Data Preparation and Splitting
# --------------------------
df = pd.read_parquet(
    r"C:\Users\forbe\OneDrive\Personal\Documents\repos\chess_data\df_features.parquet"
)

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = torch.tensor(df.iloc[:, :780].values, dtype=torch.float32).to(device)
y = torch.tensor(df["result"].values, dtype=torch.long).to(device)

dataset = TensorDataset(X, y)
total_samples = len(dataset)

train_size = int(0.8 * total_samples)
val_size = int(0.1 * total_samples)
test_size = total_samples - train_size - val_size

generator = torch.Generator().manual_seed(seed)
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=generator
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --------------------------
# Model, Initialization, and Optimizer
# --------------------------
model = CNNModel().to(device)


def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)


model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --------------------------
# Training Loop with Early Stopping
# --------------------------
best_val_loss = float("inf")
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    model.train()
    running_train_loss = 0.0
    total_train = 0

    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs_train = model(batch_x)
        loss_train = criterion(outputs_train, batch_y)
        loss_train.backward()
        optimizer.step()
        running_train_loss += loss_train.item() * batch_x.size(0)
        total_train += batch_x.size(0)

    train_loss = running_train_loss / total_train

    model.eval()
    running_val_loss = 0.0
    total_val = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            outputs_val = model(batch_x)
            loss_val = criterion(outputs_val, batch_y)
            running_val_loss += loss_val.item() * batch_x.size(0)
            total_val += batch_x.size(0)
    val_loss = running_val_loss / total_val

    print(
        f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}"
    )

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        best_model_state = model.state_dict()  # Save best state
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= PATIENCE:
        print("Early stopping triggered")
        break

model.load_state_dict(best_model_state)

# --------------------------
# Testing
# --------------------------
model.eval()
running_test_loss = 0.0
total_test = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs_test = model(batch_x)
        loss_test = criterion(outputs_test, batch_y)
        running_test_loss += loss_test.item() * batch_x.size(0)
        total_test += batch_x.size(0)

test_loss = running_test_loss / total_test
print(f"Final Test Loss: {test_loss:.4f}")

# --------------------------
# Save Checkpoint with Hyperparameters
# --------------------------
checkpoint = {
    "state_dict": model.state_dict(),
    "conv_channels": model.conv_channels,
    "kernel_size": model.kernel_size,
    "padding": model.padding,
    "fc_hidden_layers": model.fc_hidden_layers,
}
torch.save(checkpoint, "pth/chess_model.pth")

In [ ]:
# Make sure the model is in evaluation mode
model.eval()

with torch.no_grad():
    logits = model(X)  # Forward pass: get raw logits
    probabilities = torch.softmax(logits, dim=1)  # Convert logits to probabilities
    # Compute weighted score: probability of win (class 2) minus probability of loss (class 0)
    weighted_score = probabilities[:, 2] - probabilities[:, 0]

print(weighted_score)